Create matching

In [6]:
import pickle
from sklearn.utils import shuffle
import pandas as pd

with open( '/share/cp/datasets/nlp/text_classification_bias/bios/BIOS.pkl', 'rb') as file:
    bios_dicts = pickle.load(file)
df = pd.DataFrame(bios_dicts)

df.drop(['path','name','start_pos','URI','title'], axis=1, inplace=True)
df['raw_title'] = df['raw_title'].map(lambda x: x.lower())
raw_bio =[]
bio = []
for i in range(len(df)):
    raw_bio.append(df['raw'][i].replace(df['raw_title'][i],'_'))
    bio.append(df['bio'][i].replace(df['raw_title'][i],'_'))
df['raw'] = raw_bio
df['bio'] = bio
print(len(df))


361634


In [7]:
import pandas as pd
import glob

path = '/share/hel/datasets/jobiqo/talent.com/UK/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)

uk_data = pd.concat(li, axis=0, ignore_index=True)
uk_data.drop(['company','city','state','country','date','time','jobid'], axis=1, inplace=True)
uk_data['title'] = uk_data['title'].map(lambda x: x.lower())
print(len(uk_data))
uk_data.drop_duplicates(subset="description",  keep=False, inplace=True)
new_uk_data = uk_data[uk_data['title'].isin(df.raw_title.unique())]
print(len(new_uk_data))

903011
2109


In [8]:
new_df = df[df['raw_title'].isin(uk_data.title.unique())]
print(len(new_df))

320038


In [9]:
import pickle

with open('/home/deepak/RecSys2023/Dataset/bios.pkl', 'wb') as f:
    pickle.dump(new_df, f)

with open('/home/deepak/RecSys2023/Dataset/uk_jobs.pkl', 'wb') as f:
    pickle.dump(new_uk_data, f)

Split data

In [17]:
with open( '/home/deepak/RecSys2023/Dataset/uk_jobs.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs = pd.DataFrame(dicts).reset_index()
uk_jobs.drop('index', axis=1, inplace=True)

In [18]:
uk_jobs['title'].value_counts()

software engineer           522
senior software engineer    405
dentist                     279
accountant                  162
teacher                     152
architect                   146
nurse                       141
paralegal                    96
painter                      70
psychologist                 33
personal trainer             23
dietitian                    22
interior designer            18
photographer                 16
psychotherapist               4
assistant professor           4
journalist                    4
associate professor           3
attorney                      3
yoga instructor               2
model                         1
surgeon                       1
nutritionist                  1
physician                     1
Name: title, dtype: int64

In [26]:
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='physician'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='nutritionist'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='surgeon'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='model'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='yoga instructor'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='attorney'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='associate professor'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='journalist'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='assistant professor'].index)
uk_jobs = uk_jobs.drop(uk_jobs[uk_jobs['title']=='psychotherapist'].index)

In [27]:
#Load Data

from sklearn.model_selection import train_test_split

train, val_test = train_test_split(uk_jobs,random_state=100, test_size=0.30, stratify=uk_jobs['title'], shuffle=True)
test, val = train_test_split(val_test,random_state=100, test_size=0.333, stratify=val_test['title'], shuffle=True)
train = train.reset_index()
train.drop('index', axis=1, inplace=True)
val = val.reset_index()
val.drop('index', axis=1, inplace=True)
test = test.reset_index()
test.drop('index', axis=1, inplace=True)

In [29]:
len(test),len(val),len(train)

(417, 209, 1459)

In [30]:
import pickle

with open('/home/deepak/RecSys2023/Dataset/train.pkl', 'wb') as f:
    pickle.dump(train, f)

with open('/home/deepak/RecSys2023/Dataset/val.pkl', 'wb') as f:
    pickle.dump(val, f)

with open('/home/deepak/RecSys2023/Dataset/test.pkl', 'wb') as f:
    pickle.dump(test, f)

Bios with counterfactual

In [10]:
import pickle
import pandas as pd

with open( '/home/deepak/RecSys2023/Dataset/bios.pkl', 'rb') as file:
    dicts = pickle.load(file)
bios = pd.DataFrame(dicts)
#bios = pd.DataFrame(dicts).reset_index()
#bios = bios.drop('index', axis=1)

import pickle
import pandas as pd

with open( '/share/cp/datasets/nlp/text_classification_bias/bios/BIOS.pkl', 'rb') as file:
    dicts = pickle.load(file)
original_bios = pd.DataFrame(dicts)

import regex as re
from tqdm.autonotebook import tqdm

male_words = ['mr','his','he','him','himself']
female_words = ['mrs','hers','she','her','herself']

bio_list = []
counterfactual_list = []

for id in tqdm(bios.index):
    bio_string = bios['raw'][id]
    name_list = list(original_bios['name'][id])
    if len(name_list[1])==1:
        name_list[1] = name_list[1]+'.'
    if '' in name_list:
        name_list.remove('')


    bio_string = re.sub(' '.join(name_list), 'NAME',bio_string,flags=re.I)
    for name in name_list:
            bio_string = re.sub(r'\s'+name+r'\s', ' NAME ',bio_string,flags=re.I)
    if bios['gender'][id]=='M':
        bio_string = re.sub('NAME', 'Bob',bio_string)
        bio_list.append(bio_string)
        for i in range(len(male_words)):
            bio_string = re.sub(r'\s'+male_words[i]+r'\s', ' '+female_words[i]+' ',bio_string,flags=re.I)

        bio_string = re.sub('Bob', 'Alice',bio_string)
    elif bios['gender'][id]=='F':
        bio_string = re.sub('NAME', 'Alice',bio_string)
        bio_list.append(bio_string)
        for i in range(len(male_words)):
            bio_string = re.sub(r'\s'+female_words[i]+r'\s',' '+ male_words[i]+' ',bio_string,flags=re.I)

        bio_string = re.sub('Alice', 'Bob',bio_string)
    counterfactual_list.append(bio_string)

bios['bio'] = bio_list
bios['counter_bio'] = counterfactual_list
bios

/tmp/deepak/ipykernel_4019144/2747506984.py:18: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
100%|██████████| 320038/320038 [01:44<00:00, 3048.84it/s]


,raw,raw_title,gender,bio,counter_bio
0,* Nora Fisher Onar is an _ of international re...,assistant professor,F,* Alice is an _ of international relations at ...,* Bob is an _ of international relations at Ba...
2,Dr. Shawn Zelig Aster is an _ of Bible at Yesh...,assistant professor,M,Dr. Bob is an _ of Bible at Yeshiva University...,Dr. Alice is an _ of Bible at Yeshiva Universi...
3,Brittany McCandless is a _ with a print and br...,journalist,F,Alice is a _ with a print and broadcast backgr...,Bob is a _ with a print and broadcast backgrou...
4,Arturo Toledo is an Architect. He runs a bouti...,architect,M,Bob is an Architect. He runs a boutique design...,Alice is an Architect. she runs a boutique des...
5,Jan De Clercq is a solution _ working for the ...,architect,M,Bob is a solution _ working for the worldwide ...,Alice is a solution _ working for the worldwid...
...,...,...,...,...,...
361629,Fernando Quesada is an Attorney at-law admitte...,attorney,M,Bob is an Attorney at-law admitted in Mexico i...,Alice is an Attorney at-law admitted in Mexico...
361630,Jeffrey Kucik is an _ in the School of Governm...,assistant professor,M,Bob is an _ in the School of Government and Pu...,Alice is an _ in the School of Government and ...
361631,Leslie E. Seawright is an Assistant Professor ...,assistant professor,F,Alice is an Assistant Professor in the Liberal...,Bob is an Assistant Professor in the Liberal A...
361632,Paul Bird is a Chartered Accountant with over ...,accountant,M,Bob is a Chartered Accountant with over 19 yea...,Alice is a Chartered Accountant with over 19 y...


In [11]:
bios = bios.reset_index()
bios = bios.drop('index', axis=1)

with open('/home/deepak/RecSys2023/Dataset/new_bios.pkl', 'wb') as f:
    pickle.dump(bios, f)

In [2]:
import pickle
import pandas as pd
with open( '/home/deepak/RecSys2023/Dataset/val.pkl', 'rb') as file:
    dicts = pickle.load(file)
uk_jobs_val = pd.DataFrame(dicts)

In [3]:
uk_jobs_val['title'].value_counts()

software engineer           53
senior software engineer    40
dentist                     28
accountant                  16
architect                   15
teacher                     15
nurse                       14
paralegal                   10
painter                      7
psychologist                 3
interior designer            2
photographer                 2
dietitian                    2
personal trainer             2
Name: title, dtype: int64

In [4]:
uk_jobs_val['title'].unique()

array(['architect', 'software engineer', 'interior designer',
       'senior software engineer', 'dentist', 'paralegal', 'nurse',
       'accountant', 'photographer', 'teacher', 'painter', 'dietitian',
       'psychologist', 'personal trainer'], dtype=object)

Pickle to Jason File

In [1]:
import pickle
import pandas as pd

with open( '/home/deepak/RecSys2023/Dataset/new_bios.pkl', 'rb') as file:
    dicts = pickle.load(file)
bios = pd.DataFrame(dicts)
bios = pd.DataFrame(dicts).reset_index()
bios = bios.drop('index', axis=1)

home_path = '/home/deepak/RecSys2023/Dataset/BM25/'

bios['id'] = ['doc'+str(i+1) for i in range(len(bios))]

#bios_true_gender = bios.drop(['raw','counter_bio'],axis='columns')
#bios_true_gender.rename(columns = {'bio':'contents'}, inplace = True)
#bios_true_gender.to_json(home_path + 'counterfactual_gender/bios.json', orient='records')


bios_new_gender = bios.drop(['raw','bio'],axis='columns')
bios_new_gender.rename(columns = {'counter_bio':'contents'}, inplace = True)
bios_new_gender.to_json(home_path + 'original_gender/bios.json', orient='records')

GEt BM25 hits

In [2]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /home/deepak/RecSys2023/Dataset/BM25/original_gender \
  --index /home/deepak/RecSys2023/Dataset/BM25/original_gender/indexes \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-06-14 19:59:59,267 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-06-14 19:59:59,268 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-06-14 19:59:59,268 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-06-14 19:59:59,268 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: /home/deepak/RecSys2023/Dataset/BM25/original_gender
2023-06-14 19:59:59,268 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-06-14 19:59:59,268 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-06-14 19:59:59,269 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-06-14 19:59:59,269 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: en
2023-06-14 19:59:59,269 INFO 

In [3]:
!python -m pyserini.search.lucene  --index /home/deepak/RecSys2023/Dataset/BM25/original_gender/indexes --topics /home/deepak/RecSys2023/Dataset/train.tsv --output /home/deepak/RecSys2023/Dataset/BM25/original_gender/train_hits.txt  --bm25

Running /home/deepak/RecSys2023/Dataset/train.tsv topics, saving to /home/deepak/RecSys2023/Dataset/BM25/original_gender/train_hits.txt...
100%|███████████████████████████████████████| 1459/1459 [00:41<00:00, 35.13it/s]


In [4]:
!python -m pyserini.search.lucene  --index /home/deepak/RecSys2023/Dataset/BM25/original_gender/indexes --topics /home/deepak/RecSys2023/Dataset/val.tsv --output /home/deepak/RecSys2023/Dataset/BM25/original_gender/val_hits.txt  --bm25

Running /home/deepak/RecSys2023/Dataset/val.tsv topics, saving to /home/deepak/RecSys2023/Dataset/BM25/original_gender/val_hits.txt...
100%|█████████████████████████████████████████| 209/209 [00:06<00:00, 31.94it/s]


In [5]:
!python -m pyserini.search.lucene  --index /home/deepak/RecSys2023/Dataset/BM25/original_gender/indexes --topics /home/deepak/RecSys2023/Dataset/test.tsv --output /home/deepak/RecSys2023/Dataset/BM25/original_gender/test_hits.txt  --bm25

Running /home/deepak/RecSys2023/Dataset/test.tsv topics, saving to /home/deepak/RecSys2023/Dataset/BM25/original_gender/test_hits.txt...
100%|█████████████████████████████████████████| 417/417 [00:12<00:00, 33.74it/s]


Balance the candidate

In [1]:
import pickle
import pandas as pd
with open( '/home/deepak/RecSys2023/Dataset/new_bios.pkl', 'rb') as file:
    dicts = pickle.load(file)
bios = pd.DataFrame(dicts).reset_index()
bios = bios.drop('index', axis=1)
bios

,raw,raw_title,gender,bio,counter_bio
0,* Nora Fisher Onar is an _ of international re...,assistant professor,F,* Alice is an _ of international relations at ...,* Bob is an _ of international relations at Ba...
1,Dr. Shawn Zelig Aster is an _ of Bible at Yesh...,assistant professor,M,Dr. Bob is an _ of Bible at Yeshiva University...,Dr. Alice is an _ of Bible at Yeshiva Universi...
2,Brittany McCandless is a _ with a print and br...,journalist,F,Alice is a _ with a print and broadcast backgr...,Bob is a _ with a print and broadcast backgrou...
3,Arturo Toledo is an Architect. He runs a bouti...,architect,M,Bob is an Architect. He runs a boutique design...,Alice is an Architect. she runs a boutique des...
4,Jan De Clercq is a solution _ working for the ...,architect,M,Bob is a solution _ working for the worldwide ...,Alice is a solution _ working for the worldwid...
...,...,...,...,...,...
320033,Fernando Quesada is an Attorney at-law admitte...,attorney,M,Bob is an Attorney at-law admitted in Mexico i...,Alice is an Attorney at-law admitted in Mexico...
320034,Jeffrey Kucik is an _ in the School of Governm...,assistant professor,M,Bob is an _ in the School of Government and Pu...,Alice is an _ in the School of Government and ...
320035,Leslie E. Seawright is an Assistant Professor ...,assistant professor,F,Alice is an Assistant Professor in the Liberal...,Bob is an Assistant Professor in the Liberal A...
320036,Paul Bird is a Chartered Accountant with over ...,accountant,M,Bob is a Chartered Accountant with over 19 yea...,Alice is a Chartered Accountant with over 19 y...


In [8]:
i=0
for job in bios['raw_title'].unique():
    print(job)
    if i==0:
        new_bios = pd.concat([bios[(bios['gender']=='F') & (bios['raw_title']==job)].sample(n=50,random_state=1),bios[(bios['gender']=='M') & (bios['raw_title']==job)].sample(n=50,random_state=1)])
    else:
        new_bios = pd.concat([new_bios, pd.concat([bios[(bios['gender']=='F') & (bios['raw_title']==job)].sample(n=50,random_state=1),bios[(bios['gender']=='M') & (bios['raw_title']==job)].sample(n=50,random_state=1)])])
    i=i+1


assistant professor
journalist
architect
photographer
psychologist
teacher
nurse
associate professor
attorney
software engineer
painter
physician
personal trainer
surgeon
dietitian
dentist
model
psychotherapist
yoga instructor
interior designer
senior software engineer
accountant
nutritionist
paralegal


In [11]:
new_bios

,raw,raw_title,gender,bio,counter_bio
185178,Dr. Lisa Arter is an Assistant Professor of En...,assistant professor,F,Dr. Alice is an Assistant Professor of English...,Dr. Bob is an Assistant Professor of English a...
166642,"Christopher Filson, MD, MS, is an Assistant Pr...",assistant professor,F,"Alice, MD, MS, is an Assistant Professor of Ur...","Bob, MD, MS, is an Assistant Professor of Urol..."
123814,Dr. Mary A. Fox is an _ of health policy and m...,assistant professor,F,Dr. Alice is Alice _ of health policy and mana...,Dr. Bob is Bob _ of health policy and manageme...
246604,Dr. Qu Li is an _ at the Division of Psycholog...,assistant professor,F,Dr. Alice is an _ at the Division of Psycholog...,"Dr. Bob is an _ at the Division of Psychology,..."
286377,Julia Christensen is an _ of geography and pla...,assistant professor,F,Alice is an _ of geography and planning at Ros...,Bob is an _ of geography and planning at Roski...
...,...,...,...,...,...
157537,Marty Murphy is a _ with the Firm. Mr. Murphy ...,paralegal,M,Bob is a _ with the Firm. Mr. Bob graduated fr...,Alice is a _ with the Firm. Mr. Alice graduate...
96752,David Cutler is a _ with the firm's Real Estat...,paralegal,M,Bob is a _ with the firm's Real Estate group. ...,Alice is a _ with the firm's Real Estate group...
34333,Isaac Mintzer is a _ in Orlando who has worked...,paralegal,M,Bob is a _ in Orlando who has worked with many...,Alice is a _ in Orlando who has worked with ma...
79634,"Ashley Meeks is a criminal _ that lives, and w...",paralegal,M,"Bob is a criminal _ that lives, and works in O...","Alice is a criminal _ that lives, and works in..."


In [12]:
new_bios = new_bios.reset_index()
new_bios = new_bios.drop('index', axis=1)

with open('/home/deepak/RecSys2023/Dataset/new_bios.pkl', 'wb') as f:
    pickle.dump(new_bios, f)